In [1]:
import os
import io
import sys
import toytree
import toyplot.svg
import pandas as pd 
from Bio import SeqIO, AlignIO
from Bio.Align.Applications import MafftCommandline
sys.path.append(r'/davidb/yatirsolan/scripts/python/bio_utilities')
sys.path.append(r'/davidb/yatirsolan/thesis_work/figures/general')
import computational_tools
import phylogenetics
import databases
import HMM

/powerapps/share/python-anaconda-2019.10/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
working_directory = r'/davidb/yatirsolan/thesis_work/figures/novel_insights/T4SSB_figure/'
full_genomes_file = 'species_full_genomes.faa'
full_genomes_dict = SeqIO.to_dict(SeqIO.parse(os.path.join(working_directory, full_genomes_file), 'fasta'))

In [6]:
! /davidb/local/bin/hmmsearch -o /dev/null --tblout T4SSB_HMMs_vs_species_full_genomes.hmmsearch --notextw -E 1e-4 --cpu 4 /davidb/yatirsolan/secretion_systems/T4SS/hmm_repository/T4SSB.Hmm /davidb/yatirsolan/thesis_work/figures/novel_insights/T4SSB_figure/species_full_genomes.faa

In [7]:
HMM.hmmsearch_to_tsv(r'/davidb/yatirsolan/thesis_work/figures/novel_insights/T4SSB_figure/T4SSB_HMMs_vs_species_full_genomes.hmmsearch')

'/davidb/yatirsolan/thesis_work/figures/novel_insights/T4SSB_figure/T4SSB_HMMs_vs_species_full_genomes.hmmsearch.tsv'

In [12]:
GCA_to_taxid_dict = dict(pd.read_table(r'/davidb/bio_db/NCBI/WGS/latest.assembly_summary_genbank.wLineage.no_fungi_metazoa_viridiplantae.tab', usecols=['# assembly_accession', 'taxid']).values)
GCA_to_taxid_dict = {k:str(v) for k, v in GCA_to_taxid_dict.items()}

In [9]:
species_to_include = ['Legionella oakridgensis',
                      'Legionella erythra',
                      'Legionella hackeliae',
                      'Legionella pneumophila subsp. pneumophila str. Philadelphia 1',
                      'Legionella moravica',
                      'Legionella anisa',
                      'Tatlockia micdadei',
                      'Fluoribacter dumoffii NY 23',
                      'Legionella longbeachae',
                      'Piscirickettsia salmonis',
                      'Piscirickettsia litoralis',
                      'Pseudomonas amygdali pv. sesami',
                      'Pseudomonas monteilii',
                      'Pseudomonas parafulva NBRC 16636 = DSM 17004',
                      'Pseudomonas putida B001',
                      'Coxiella burnetii RSA 493',
                      'Rickettsiella grylli',
                      'Legionella geestiana']

species_to_exclude = []

In [10]:
hmmsrch_df = pd.read_table(os.path.join(working_directory, 'T4SSB_HMMs_vs_species_full_genomes.hmmsearch.tsv'), 
                           usecols=['targetName', 'queryName', 'queryAccession', 'E-value'])
hmmsrch_df

,targetName,queryName,queryAccession,E-value
0,Legionella_shakespearei_DSM_23087|SAMN04274794...,DotA,Leg_DotA,0.000000e+00
1,Legionella_moravica|SAMN04274785|GCA_001467865...,DotA,Leg_DotA,0.000000e+00
2,Fluoribacter_bozemanae|SAMN04274766|GCA_001467...,DotA,Leg_DotA,0.000000e+00
3,Legionella_parisiensis|SAMN04274788|GCA_001467...,DotA,Leg_DotA,0.000000e+00
4,Fluoribacter_dumoffii_NY_23|SAMN04274771|GCA_0...,DotA,Leg_DotA,0.000000e+00
...,...,...,...,...
7217,Pseudomonas_amygdali_pv._sesami|SAMN03976303|G...,DotC,Pisc_DotC,1.500000e-30
7218,Pseudomonas_monteilii|SAMN11579699|GCA_0098313...,DotC,Pisc_DotC,8.000000e-30
7219,Pseudomonas_putida_B001|SAMEA2272788|GCA_00028...,DotC,Pisc_DotC,8.000000e-30
7220,Piscirickettsia_litoralis|SAMN05601431|GCA_001...,DotC,Pisc_DotC,2.200000e-27


In [13]:
def edit_seq(seq):
    seq.name = seq.name.split('|')[0]
    seq.name = '_'.join(seq.name.split('_')[:2:])
    seq.id = seq.id.split('|')[0]
    seq.id = '_'.join(seq.id.split('_')[:2:])
    seq.description = str()
    return seq

hmmsrch_df['GCA'] = hmmsrch_df.targetName.map(databases.gca_from_accession)
hmmsrch_df['tax'] = hmmsrch_df.GCA.map(GCA_to_taxid_dict)
hmmsrch_df['tax'] = hmmsrch_df.tax.map(phylogenetics.taxid_to_name)
hmmsrch_df = hmmsrch_df[hmmsrch_df.tax.isin(species_to_include)]

hmmsrch_df = hmmsrch_df.sort_values('E-value')
hmmsrch_df = hmmsrch_df.drop_duplicates('targetName', keep='first')
hmmsrch_df = hmmsrch_df.sort_values(['queryName', 'tax', 'E-value'])
hmmsrch_df = hmmsrch_df.drop_duplicates(subset=['queryName', 'tax'], keep='first')
hmmsrch_df = hmmsrch_df.sort_values(['queryName', 'tax'])

hmmsrch_df['seq'] = hmmsrch_df.targetName.map(full_genomes_dict)
hmmsrch_df['seq'] = hmmsrch_df.seq.map(edit_seq)

hmmsrch_df

,targetName,queryName,queryAccession,E-value,GCA,tax,seq
2829,Coxiella_burnetii_RSA_493|SAMN02603970|GCA_000...,DotA,Cox_DotA,1.100000e-280,GCA_000007765.2,Coxiella burnetii RSA 493,"(M, K, K, L, V, S, S, L, L, A, S, I, S, L, F, ..."
4,Fluoribacter_dumoffii_NY_23|SAMN04274771|GCA_0...,DotA,Leg_DotA,0.000000e+00,GCA_001467605.1,Fluoribacter dumoffii NY 23,"(M, K, R, L, L, G, T, L, L, L, L, L, F, P, G, ..."
10,Legionella_anisa|SAMN04274764|GCA_001467525.1|...,DotA,Leg_DotA,0.000000e+00,GCA_001467525.1,Legionella anisa,"(M, N, K, L, L, V, T, L, L, L, L, V, C, P, G, ..."
8,Legionella_erythra|SAMN04274772|GCA_001467615....,DotA,Leg_DotA,0.000000e+00,GCA_001467615.1,Legionella erythra,"(M, N, K, I, V, S, T, L, L, M, T, L, F, P, A, ..."
37,Legionella_geestiana|SAMEA103899863|GCA_900452...,DotA,Leg_DotA,0.000000e+00,GCA_900452375.1,Legionella geestiana,"(M, K, K, Y, I, L, G, L, L, S, A, L, M, P, V, ..."
...,...,...,...,...,...,...,...
2039,Legionella_moravica|SAMN04274785|GCA_001467865...,IcmX,Leg_IcmX,5.400000e-184,GCA_001467865.1,Legionella moravica,"(M, K, L, L, S, K, C, S, L, M, T, L, L, C, L, ..."
2068,Legionella_oakridgensis|SAMN04274787|GCA_00146...,IcmX,Leg_IcmX,1.900000e-133,GCA_001467925.1,Legionella oakridgensis,"(M, K, I, I, G, R, S, L, F, A, C, L, L, S, M, ..."
2058,Legionella_pneumophila_subsp._pneumophila_str....,IcmX,Leg_IcmX,7.900000e-167,GCA_000008485.1,Legionella pneumophila subsp. pneumophila str....,"(M, K, V, L, P, K, L, A, L, A, N, L, V, C, F, ..."
4735,Rickettsiella_grylli|SAMN02435840|GCA_00016829...,IcmX,Cox_IcmX,8.400000e-153,GCA_000168295.1,Rickettsiella grylli,"(M, M, Y, P, S, K, I, K, K, I, K, I, H, F, I, ..."


In [14]:
proteins_for_tree = set.intersection(*[set(prots) for prots in hmmsrch_df.groupby('tax').queryName.unique()])
proteins_to_exclude = {'IcmH_DotU'}
proteins_for_tree = proteins_for_tree.union(proteins_to_exclude)
hmmsrch_df = hmmsrch_df[hmmsrch_df.queryName.isin(proteins_for_tree)]
hmmsrch_df

,targetName,queryName,queryAccession,E-value,GCA,tax,seq
2829,Coxiella_burnetii_RSA_493|SAMN02603970|GCA_000...,DotA,Cox_DotA,1.100000e-280,GCA_000007765.2,Coxiella burnetii RSA 493,"(M, K, K, L, V, S, S, L, L, A, S, I, S, L, F, ..."
4,Fluoribacter_dumoffii_NY_23|SAMN04274771|GCA_0...,DotA,Leg_DotA,0.000000e+00,GCA_001467605.1,Fluoribacter dumoffii NY 23,"(M, K, R, L, L, G, T, L, L, L, L, L, F, P, G, ..."
10,Legionella_anisa|SAMN04274764|GCA_001467525.1|...,DotA,Leg_DotA,0.000000e+00,GCA_001467525.1,Legionella anisa,"(M, N, K, L, L, V, T, L, L, L, L, V, C, P, G, ..."
8,Legionella_erythra|SAMN04274772|GCA_001467615....,DotA,Leg_DotA,0.000000e+00,GCA_001467615.1,Legionella erythra,"(M, N, K, I, V, S, T, L, L, M, T, L, F, P, A, ..."
37,Legionella_geestiana|SAMEA103899863|GCA_900452...,DotA,Leg_DotA,0.000000e+00,GCA_900452375.1,Legionella geestiana,"(M, K, K, Y, I, L, G, L, L, S, A, L, M, P, V, ..."
...,...,...,...,...,...,...,...
4634,Pseudomonas_monteilii|SAMN11579699|GCA_0098313...,IcmT,Cox_IcmT,6.500000e-06,GCA_009831395.1,Pseudomonas monteilii,"(M, A, T, G, R, Y, H, W, S, R, C, A, P, P, A, ..."
4632,Pseudomonas_parafulva_NBRC_16636___DSM_17004|S...,IcmT,Cox_IcmT,1.100000e-06,GCA_000730645.1,Pseudomonas parafulva NBRC 16636 = DSM 17004,"(M, A, T, G, R, Y, H, W, S, R, S, A, P, P, A, ..."
4635,Pseudomonas_putida_B001|SAMEA2272788|GCA_00028...,IcmT,Cox_IcmT,6.500000e-06,GCA_000285395.1,Pseudomonas putida B001,"(M, A, T, G, R, Y, H, W, S, R, C, A, P, P, A, ..."
4589,Rickettsiella_grylli|SAMN02435840|GCA_00016829...,IcmT,Cox_IcmT,1.100000e-43,GCA_000168295.1,Rickettsiella grylli,"(M, S, R, V, P, T, A, A, H, W, R, D, S, A, R, ..."


In [15]:
species = set(hmmsrch_df.tax.unique())
with_geest = {k:set(v).symmetric_difference(species) for k, v in dict(hmmsrch_df.groupby('queryName').tax.unique()).items()}
with_geest

{'DotA': set(),
 'DotB': set(),
 'DotC': set(),
 'DotD': set(),
 'DotK': set(),
 'IcmB_DotO': set(),
 'IcmC': set(),
 'IcmE_DotG': set(),
 'IcmG_DotF': set(),
 'IcmH_DotU': {'Rickettsiella grylli'},
 'IcmJ_DotN': set(),
 'IcmK_DotH': set(),
 'IcmL_DotI': set(),
 'IcmO_DotL': set(),
 'IcmP_DotM': set(),
 'IcmT': set()}

Creating the alignment of DotU in ordet to find the root.

In [14]:
DotU_fasta = hmmsrch_df[hmmsrch_df.queryName == 'IcmH_DotU'].seq.to_list()
a_tumafaciens_DotU = SeqIO.read(r'/davidb/yatirsolan/thesis_work/figures/novel_insights/T4SSB_figure/full_genomes/Agrobacterium_tumefaciens/Agrobacterium_tumefaciens_DotU.faa', 'fasta')
a_tumafaciens_DotU.name = a_tumafaciens_DotU.name.split('|')[0]
a_tumafaciens_DotU.id = a_tumafaciens_DotU.name
a_tumafaciens_DotU.description = str()
DotU_fasta.append(a_tumafaciens_DotU)
fasta_file = os.path.join(working_directory, 'IcmH_DotU.faa')
SeqIO.write(DotU_fasta, fasta_file, 'fasta')
mafft_cline = MafftCommandline(cmd=computational_tools.mafft_paths(algorithm='linsi'), input=fasta_file)
stdout, stderr = mafft_cline()
alignment = AlignIO.read(io.StringIO(stdout), 'fasta')
AlignIO.write(alignment, os.path.join(working_directory, 'IcmH_DotU.aln'), 'fasta')
os.system(f"{computational_tools.tree_path(algorithm='fast_tree')} {os.path.join(working_directory, 'IcmH_DotU.aln')} > {os.path.join(working_directory, 'IcmH_DotU.nw')}")

0

In [15]:
hmmsrch_df = hmmsrch_df[~(hmmsrch_df.queryName.isin(proteins_to_exclude))]
MSA_list = list()
for protein, sub_df in hmmsrch_df.groupby('queryName'):
    sub_df = sub_df.sort_values('tax')
    fasta_file = os.path.join(working_directory, f'{protein}.faa')
    SeqIO.write(sub_df.seq, fasta_file, 'fasta')
    mafft_cline = MafftCommandline(cmd=computational_tools.mafft_paths(algorithm='linsi'), input=fasta_file)
    stdout, stderr = mafft_cline()
    alignment = AlignIO.read(io.StringIO(stdout), 'fasta')
    AlignIO.write(alignment, fasta_file.replace('.faa', '.aln'), 'fasta')
    MSA_list.append(alignment)

In [16]:
for MSA in MSA_list:
    try:
        cncatntd_MSA += MSA
    except NameError:
        cncatntd_MSA = MSA
        
for MSA in cncatntd_MSA:
    MSA.description = str()
    
concatenated_file = os.path.join(working_directory, 'concatenated.aln')
AlignIO.write(cncatntd_MSA, concatenated_file, 'fasta')
os.system(f"{computational_tools.tree_path(algorithm='fast_tree')} {concatenated_file} > {os.path.join(working_directory, 'concatenated.nw')}")

0

In [3]:
def determine_color(label):
    if 'Legionella' in label or 'Fluoribacter' in label or 'Tatlockia' in label :
        return '#2166ac'
    elif 'Coxiella' in label or 'Aquicella' in label or 'Rickettsiella' in label:
        return '#1b7837'
    elif 'Pseu' in label:
        return '#d6557c'
    elif 'Piscirickettsia' in label:
        return '#9B14F0'
    return 'black'

def f(label):
   start = "\x1B[3m"
   end = "\x1B[0m"   
   return start + label + end

In [49]:
tree = toytree.tree(os.path.join(working_directory, 'IcmH_DotU.nw'))
tree = toytree.tree(os.path.join(working_directory, 'IcmH_DotU.raxml.nw'))
tree = tree.root(names=['Agrobacterium_tumefaciens'])
tip_labels = [' '.join(label.split('_')[:2:]).replace('Tatlockia', 'Legionella').replace('Fluoribacter', 'Legionella') for label in tree.get_tip_labels()]
tip_labels = ["<i>{}</i>".format(label) for label in tip_labels]
canvas, axes, mark = \
tree.draw(layout='r',
           height=400, 
           width=500,
           tip_labels_align=True,
           tip_labels_colors = list(map(determine_color, tree.get_tip_labels())),
           tip_labels=tip_labels,
           tip_labels_style={'font-size':'11px', '-toyplot-anchor-shift':'10px'},
           edge_align_style={'stroke':'grey', 'stroke-width':.5, 'stroke-dasharray':'3,1'}) 

toyplot.html.render(canvas, os.path.join(working_directory, 'IcmH_DotU_raxml_toytree_italic.html'))
# canvas

In [5]:
tree = toytree.tree(os.path.join(working_directory, 'concatenated.nw'))
tree = toytree.tree(os.path.join(working_directory, 'concatenated.raxml.nw'))
tree = tree.root(names=list(filter(lambda tip:'Piscirickettsia' in tip, tree.get_tip_labels())))
novel_species = dict(filter(lambda a:a[1] in ['Pseudomonas_amygdali', 'Pseudomonas_parafulva'], tree.get_node_dict().items()))
canvas, axes, mark = \
tree.draw(layout='r',
           height=400, 
           width=600,
           node_colors='#d53e4f',
           tip_labels_align=True,
           tip_labels_colors = list(map(determine_color, tree.get_tip_labels())),
           tip_labels=[' '.join(label.split('_')[:2:]).replace('Tatlockia', 'Legionella').replace('Fluoribacter', 'Legionella') for label in tree.get_tip_labels()],
           node_sizes=[8 if i in novel_species.keys() else 0 for i in tree.get_node_values('idx', True, True)], # size of dash, spacing of dashes
           tip_labels_style={'font-size': '11px', '-toyplot-anchor-shift': '10px'},
           edge_align_style={'stroke': 'grey', 'stroke-width': .5, 'stroke-dasharray': '3,1'}) 

toyplot.svg.render(canvas, os.path.join(working_directory, 'figure5_a.svg'))

<svg class="toyplot-canvas-Canvas" height="400.0px" id="t842f0b4e3d0f49a68731559fc4830bc3" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 400.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Legionella dumoffii Legionella anisa Legionella longbeachae Legionella moravica Legionella pneumophila Legionella micdadei Legionella hackeliae Legionella erythra Legionella geestiana Legionella oakridgensis Rickettsiella grylli Coxiella burnetii Pseudomonas monteilii Pseudomonas putida Pseudomonas parafulva Pseudomonas amygdali Piscirickettsia litoralis Piscirickettsia salmonis